# neural network

### Import libraries

In [1]:
import sys, os

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import time
import copy

In [3]:
from src.neural_network import utils
from src.neural_network import resnet
from torchvision import models

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, 
                 hidden_size3, hidden_size4, hidden_size5, 
                 hidden_size6, hidden_size7, hidden_size8,
                 hidden_size9, hidden_size10, hidden_size11, 
                 hidden_size12, hidden_size13, hidden_size14, 
                 hidden_size15, hidden_size16, hidden_size17, 
                 output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, hidden_size3),
            nn.ReLU(),
            nn.Linear(hidden_size3, hidden_size4),
            nn.ReLU(),
            nn.Linear(hidden_size4, hidden_size5),
            nn.ReLU(),
            nn.Linear(hidden_size5, hidden_size6),
            nn.ReLU(),
            nn.Linear(hidden_size6, hidden_size7),
            nn.ReLU(),
            nn.Linear(hidden_size7, hidden_size8),
            nn.ReLU(),
            nn.Linear(hidden_size8, hidden_size9),
            nn.ReLU(),
            nn.Linear(hidden_size9, hidden_size10),
            nn.ReLU(),
            nn.Linear(hidden_size10, hidden_size11),
            nn.ReLU(),
            nn.Linear(hidden_size11, hidden_size12),
            nn.ReLU(),
            nn.Linear(hidden_size12, hidden_size13),
            nn.ReLU(),
            nn.Linear(hidden_size13, hidden_size14),
            nn.ReLU(),
            nn.Linear(hidden_size14, hidden_size15),
            nn.ReLU(),
            nn.Linear(hidden_size15, hidden_size16),
            nn.ReLU(),
            nn.Linear(hidden_size16, hidden_size17),
            nn.ReLU(),
            nn.Linear(hidden_size17, output_size)
        )
        
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

### Configuration

In [5]:
#
# cpu or gpu
#
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

#
# dataset
#
train_dataset = "./dataset/merged_train_bin_10"
#train_dataset = "./dataset/merged_train_63"
test_dataset = "./dataset/merged_test_bin_10"

# output of networks
classes = 10

#
# net
#
# input of networks
channels = 3
img_pixels = (200,200)

# the test is much faster than train, but not negligable (say <15% of the whole time)
# 74 training imgs, 10 test imgs

# 0, 1 class has 100%, very fast
#net = MLP(channels * img_pixels[0] * img_pixels[1], 512, 512,
#          512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
#          512, 512, 512, 512, 512, classes)


# 5, from 0.005 to 0.02, 12s per epoch
#net = models.resnet18(num_classes=classes)
#net = models.resnet34(num_classes=classes)
#net = models.resnet50(num_classes=classes)
#net = models.resnet101(num_classes=classes)
#net = models.resnet152(num_classes=classes)

# 0, 1 class has 100%, 4s per epoch
#net = models.alexnet(num_classes=classes)

# 10, all near 0.01, 2min+ per epoch
#net = models.vgg19(num_classes=classes)

# 10, all near 0.01, 2min+ per epoch
net = models.vgg19_bn(num_classes=classes)

# 7, from 0.0096 to 0.012, 6s per epoch
#net = models.squeezenet1_0(num_classes=classes)

# 6, from 0.005 to 0.015, 100s per epoch
#net = models.densenet161(num_classes=classes)

# tricky to run, maybe later (maybe never)
# depends on scipy
# can not run on (256, 256) pixel, but work at (512, 512)
# 'InceptionOutputs' object has no attribute 'log_softmax'
#net = models.inception_v3(num_classes=classes)

# tricky to run, maybe later (maybe never)
# can not run on (256, 256) pixel, but work at (512, 512)
#net = models.googlenet(num_classes=classes)

# 10, all near 0.01, 3s per epoch
#net = models.shufflenet_v2_x1_0(num_classes=classes)

# 10, all exactly 0.01, 8s per epoch
#net = models.mobilenet_v2(num_classes=classes)

# 10, all near 0.01, 30s per epoch
#net = models.resnext50_32x4d(num_classes=classes)

# 0, 1 class takes 0.97, 1min per epoch
#net = models.wide_resnet50_2(num_classes=classes)

# 10, all exactly 0.01, 8s per epoch
#net = models.mnasnet1_0(num_classes=classes)

#
# training detail
#
# CONFIRM: whether lr is dynamically determined by pytorch
lr = 0.001
num_epochs = 50
batch_size = 128

#
# misc
#
#print(net)
utils.display_num_param(net)

model_save_name = "%s_%s_%s" % (num_epochs, net.__class__.__name__, os.path.basename(train_dataset))
print(model_save_name)

cuda
There are 139622218 (139.62 million) parameters in this neural network
50_VGG_merged_train_bin_10


### Setting train & eval & test dataset

In [6]:
import torchvision
from torchvision import transforms

transform = transforms.Compose([
    # TODO: check this more later
    #transforms.Grayscale(num_output_channels=3),
    transforms.Resize(img_pixels),
    transforms.ToTensor()])

img_data_train = torchvision.datasets.ImageFolder(root=train_dataset, transform=transform)
print('train data len:', len(img_data_train))
data_loader_train = torch.utils.data.DataLoader(img_data_train, batch_size=batch_size,shuffle=True)
print(len(data_loader_train))

img_data_val = torchvision.datasets.ImageFolder(root=test_dataset, transform=transform)
print(len(img_data_val))
data_loader_val = torch.utils.data.DataLoader(img_data_val, batch_size=batch_size,shuffle=True)
print(len(data_loader_val))

img_data_test = torchvision.datasets.ImageFolder(root=test_dataset, transform=transform)
data_loader_test = torch.utils.data.DataLoader(img_data_test, batch_size=1,shuffle=True)

dataloaders = {}
dataloaders['train'] = data_loader_train
dataloaders['val'] = data_loader_val

train data len: 32818
257
5182
41


### Helper function to training

In [7]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    last = since
    time_elapsed = since

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            time_elapsed = time.time() - last
            last = time.time()
            
            print('{} Loss: {:.4f} Acc: {:.4f} Time: {:.0f}m {:.0f}s'.format(phase, epoch_loss, epoch_acc, time_elapsed // 60, time_elapsed % 60))

            # deep copy the modeltopk
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

### Training

In [ ]:
net = net.to(device)
criterion = nn.CrossEntropyLoss()
# use ADAM, better than SGD
optimizer=torch.optim.Adam(net.parameters(), lr)
net, val_acc_history = train_model(net, dataloaders, criterion, optimizer, num_epochs)

Epoch 0/49
----------
train Loss: 2.9049 Acc: 0.2599 Time: 7m 11s
val Loss: 1.9587 Acc: 0.2995 Time: 0m 26s

Epoch 1/49
----------
train Loss: 1.9471 Acc: 0.2701 Time: 7m 8s
val Loss: 1.9605 Acc: 0.3001 Time: 0m 26s

Epoch 2/49
----------
train Loss: 1.9195 Acc: 0.2757 Time: 7m 6s
val Loss: 1.9093 Acc: 0.3173 Time: 0m 26s

Epoch 3/49
----------
train Loss: 1.9040 Acc: 0.2795 Time: 7m 7s
val Loss: 1.9106 Acc: 0.3284 Time: 0m 26s

Epoch 4/49
----------
train Loss: 1.8722 Acc: 0.2955 Time: 7m 10s
val Loss: 1.8490 Acc: 0.3568 Time: 0m 26s

Epoch 5/49
----------
train Loss: 1.8388 Acc: 0.3102 Time: 7m 8s
val Loss: 1.8503 Acc: 0.3676 Time: 0m 26s

Epoch 6/49
----------
train Loss: 1.8528 Acc: 0.3038 Time: 7m 6s
val Loss: 1.9073 Acc: 0.2997 Time: 0m 26s

Epoch 7/49
----------
train Loss: 1.8676 Acc: 0.2898 Time: 7m 10s
val Loss: 1.7635 Acc: 0.3638 Time: 0m 26s

Epoch 8/49
----------
train Loss: 1.8132 Acc: 0.3156 Time: 7m 8s
val Loss: 1.8358 Acc: 0.3688 Time: 0m 26s

Epoch 9/49
----------
tra

### Save Model

In [ ]:
!mkdir -p ./saved_models
torch.save(net.state_dict(), os.path.join("./saved_models/", model_save_name))

### Evaluation on test data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def show_prob_age(label, p):

    p=p.data.squeeze().numpy()

    ft=15
    #p=p.data.squeeze().numpy()
    y_pos = np.arange(len(p))*1.2
    target=2
    width=0.9
    col= 'blue'
    #col='darkgreen'

    plt.rcdefaults()
    fig, ax = plt.subplots()

    # the plot
    ax.barh(y_pos, p, width , align='center', color=col)

    ax.set_xlim([0, 1.3])
    #ax.set_ylim([-0.8, len(p)*1.2-1+0.8])

    # y label
    ax.set_yticks(y_pos)
    ax.set_yticklabels(label, fontsize=ft)
    ax.invert_yaxis()
    #ax.set_xlabel('Performance')
    #ax.set_title('How fast do you want to go today?')

    # x label
    ax.set_xticklabels([])
    ax.set_xticks([])
    #x_pos=np.array([0, 0.25 , 0.5 , 0.75 , 1])
    #ax.set_xticks(x_pos)
    #ax.set_xticklabels( [0, 0.25 , 0.5 , 0.75 , 1] , fontsize=15)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_linewidth(4)

    for i in range(len(p)):
        str_nb="{0:.2f}".format(p[i])
        ax.text( p[i] + 0.05 , y_pos[i] ,str_nb ,
                 horizontalalignment='left', #verticalalitopkgnment='center',
                 transform=ax.transData, color= col,fontsize=ft)

    plt.show()


In [ ]:
# switch to eval mode
net.eval()

label_dict = dict(map(reversed, img_data_test.class_to_idx.items()))

dataiter = iter(data_loader_test)
input, label = dataiter.next()
input, label = input.to(device),label.to(device)

# display tensor

#print(input.size())
image = torch.tensor(input.tolist()[0])
trans = transforms.ToPILImage()
print("Original label:", label_dict[label.tolist()[0]])
#print(image.size())
plt.imshow(trans(image))
output = net(input)
probs= F.softmax(output, dim=1)
top10 = torch.topk(probs[0], 10)
top10value = top10.values
#print(top10value)
top10ages = [ label_dict[a] for a in top10.indices.tolist() ]
#print(top10value, top10ages)
show_prob_age(top10ages, top10value.cpu())

### result of 200 epoches, resnet 151, 63 classes, each 173 samples

```text
Epoch 0/199
----------
train Loss: 4.3169 Acc: 0.0181 Time: 2m 47s
val Loss: 4.3966 Acc: 0.0159 Time: 0m 6s

Epoch 1/199
----------
train Loss: 4.1316 Acc: 0.0256 Time: 2m 47s
val Loss: 4.1145 Acc: 0.0233 Time: 0m 6s

Epoch 2/199
----------
train Loss: 4.0982 Acc: 0.0297 Time: 2m 46s
val Loss: 4.1161 Acc: 0.0265 Time: 0m 6s

Epoch 3/199
----------
train Loss: 4.0608 Acc: 0.0307 Time: 2m 46s
val Loss: 134.2509 Acc: 0.0190 Time: 0m 6s

Epoch 4/199
----------
train Loss: 3.9842 Acc: 0.0342 Time: 2m 46s
val Loss: 4.0242 Acc: 0.0222 Time: 0m 6s

Epoch 5/199
----------
train Loss: 3.8732 Acc: 0.0459 Time: 2m 46s
val Loss: 4.4584 Acc: 0.0212 Time: 0m 6s

Epoch 6/199
----------
train Loss: 3.7959 Acc: 0.0515 Time: 2m 46s
val Loss: 4.5372 Acc: 0.0423 Time: 0m 6s

Epoch 7/199
----------
train Loss: 3.7056 Acc: 0.0553 Time: 2m 45s
val Loss: 3.7435 Acc: 0.0529 Time: 0m 6s

Epoch 8/199
----------
train Loss: 3.6168 Acc: 0.0644 Time: 2m 46s
val Loss: 3.7011 Acc: 0.0667 Time: 0m 6s

Epoch 9/199
----------
train Loss: 3.5561 Acc: 0.0649 Time: 2m 46s
val Loss: 4.6276 Acc: 0.0402 Time: 0m 6s

Epoch 10/199
----------
train Loss: 3.5105 Acc: 0.0679 Time: 2m 46s
val Loss: 4.1805 Acc: 0.0423 Time: 0m 6s

Epoch 11/199
----------
train Loss: 3.4603 Acc: 0.0676 Time: 2m 47s
val Loss: 3.7268 Acc: 0.0582 Time: 0m 6s

Epoch 12/199
----------
train Loss: 3.3906 Acc: 0.0799 Time: 2m 46s
val Loss: 4.2390 Acc: 0.0434 Time: 0m 6s

Epoch 13/199
----------
train Loss: 3.3330 Acc: 0.0825 Time: 2m 45s
val Loss: 5.2585 Acc: 0.0317 Time: 0m 6s

Epoch 14/199
----------
train Loss: 3.2923 Acc: 0.0885 Time: 2m 45s
val Loss: 3.8341 Acc: 0.0519 Time: 0m 6s

Epoch 15/199
----------
train Loss: 3.2376 Acc: 0.0918 Time: 2m 46s
val Loss: 3.7672 Acc: 0.0677 Time: 0m 6s

Epoch 16/199
----------
train Loss: 3.1513 Acc: 0.0986 Time: 2m 45s
val Loss: 3.7788 Acc: 0.0677 Time: 0m 6s

Epoch 17/199
----------
train Loss: 3.0867 Acc: 0.1052 Time: 2m 45s
val Loss: 4.5777 Acc: 0.0402 Time: 0m 6s

Epoch 18/199
----------
train Loss: 3.0265 Acc: 0.1138 Time: 2m 45s
val Loss: 4.7459 Acc: 0.0677 Time: 0m 6s

Epoch 19/199
----------
train Loss: 2.9887 Acc: 0.1196 Time: 2m 45s
val Loss: 4.0717 Acc: 0.0635 Time: 0m 6s

Epoch 20/199
----------
train Loss: 2.9233 Acc: 0.1290 Time: 2m 46s
val Loss: 4.2541 Acc: 0.0571 Time: 0m 6s

Epoch 21/199
----------
train Loss: 2.8537 Acc: 0.1394 Time: 2m 45s
val Loss: 4.4596 Acc: 0.0519 Time: 0m 6s

Epoch 22/199
----------
train Loss: 2.7973 Acc: 0.1507 Time: 2m 45s
val Loss: 4.1456 Acc: 0.0772 Time: 0m 6s

Epoch 23/199
----------
train Loss: 2.7019 Acc: 0.1713 Time: 2m 47s
val Loss: 4.0666 Acc: 0.0667 Time: 0m 6s

Epoch 24/199
----------
train Loss: 2.6393 Acc: 0.1872 Time: 2m 46s
val Loss: 5.6378 Acc: 0.0487 Time: 0m 6s

Epoch 25/199
----------
train Loss: 2.6233 Acc: 0.1918 Time: 2m 46s
val Loss: 5.5191 Acc: 0.0370 Time: 0m 6s

Epoch 26/199
----------
train Loss: 2.6325 Acc: 0.1963 Time: 2m 47s
val Loss: 4.1314 Acc: 0.0698 Time: 0m 6s

Epoch 27/199
----------
train Loss: 2.4039 Acc: 0.2446 Time: 2m 47s
val Loss: 4.7814 Acc: 0.0593 Time: 0m 6s

Epoch 28/199
----------
train Loss: 2.2785 Acc: 0.2740 Time: 2m 47s
val Loss: 4.8640 Acc: 0.0603 Time: 0m 6s

Epoch 29/199
----------
train Loss: 2.1767 Acc: 0.3047 Time: 2m 47s
val Loss: 4.9726 Acc: 0.0614 Time: 0m 6s

Epoch 30/199
----------
train Loss: 2.0525 Acc: 0.3387 Time: 2m 46s
val Loss: 5.5820 Acc: 0.0667 Time: 0m 6s

Epoch 31/199
----------
train Loss: 1.9613 Acc: 0.3703 Time: 2m 46s
val Loss: 6.5476 Acc: 0.0677 Time: 0m 6s

Epoch 32/199
----------
train Loss: 1.8263 Acc: 0.3971 Time: 2m 46s
val Loss: 5.8180 Acc: 0.0720 Time: 0m 6s

Epoch 33/199
----------
train Loss: 1.6922 Acc: 0.4521 Time: 2m 45s
val Loss: 6.6958 Acc: 0.0646 Time: 0m 6s

Epoch 34/199
----------
train Loss: 1.5263 Acc: 0.5002 Time: 2m 46s
val Loss: 6.7573 Acc: 0.0825 Time: 0m 6s

Epoch 35/199
----------
train Loss: 1.4059 Acc: 0.5412 Time: 2m 46s
val Loss: 6.5108 Acc: 0.0720 Time: 0m 6s

Epoch 36/199
----------
train Loss: 1.2236 Acc: 0.5990 Time: 2m 47s
val Loss: 8.2109 Acc: 0.0593 Time: 0m 6s

Epoch 37/199
----------
train Loss: 1.0961 Acc: 0.6455 Time: 2m 46s
val Loss: 7.9884 Acc: 0.0593 Time: 0m 6s

Epoch 38/199
----------
train Loss: 1.0285 Acc: 0.6746 Time: 2m 45s
val Loss: 9.4791 Acc: 0.0603 Time: 0m 6s

Epoch 39/199
----------
train Loss: 0.9111 Acc: 0.7061 Time: 2m 46s
val Loss: 8.3382 Acc: 0.0624 Time: 0m 6s

Epoch 40/199
----------
train Loss: 0.8652 Acc: 0.7355 Time: 2m 45s
val Loss: 9.2519 Acc: 0.0582 Time: 0m 6s

Epoch 41/199
----------
train Loss: 0.7337 Acc: 0.7712 Time: 2m 46s
val Loss: 8.6766 Acc: 0.0794 Time: 0m 6s

Epoch 42/199
----------
train Loss: 0.6474 Acc: 0.8044 Time: 2m 48s
val Loss: 10.5510 Acc: 0.0593 Time: 0m 6s

Epoch 43/199
----------
train Loss: 0.4735 Acc: 0.8545 Time: 2m 46s
val Loss: 9.5515 Acc: 0.0656 Time: 0m 6s

Epoch 44/199
----------
train Loss: 0.4361 Acc: 0.8748 Time: 2m 46s
val Loss: 10.8023 Acc: 0.0730 Time: 0m 6s

Epoch 45/199
----------
train Loss: 0.4759 Acc: 0.8607 Time: 2m 47s
val Loss: 9.9721 Acc: 0.0709 Time: 0m 6s

Epoch 46/199
----------
train Loss: 0.4129 Acc: 0.8788 Time: 2m 46s
val Loss: 10.1475 Acc: 0.0730 Time: 0m 6s

Epoch 47/199
----------
train Loss: 0.3633 Acc: 0.8992 Time: 2m 46s
val Loss: 9.9304 Acc: 0.0878 Time: 0m 6s

Epoch 48/199
----------
train Loss: 0.3455 Acc: 0.9004 Time: 2m 45s
val Loss: 10.9654 Acc: 0.0667 Time: 0m 6s

Epoch 49/199
----------
train Loss: 0.2819 Acc: 0.9238 Time: 2m 46s
val Loss: 11.8058 Acc: 0.0614 Time: 0m 6s

Epoch 50/199
----------
train Loss: 0.2347 Acc: 0.9363 Time: 2m 45s
val Loss: 11.0187 Acc: 0.0698 Time: 0m 6s

Epoch 51/199
----------
train Loss: 0.1696 Acc: 0.9599 Time: 2m 46s
val Loss: 11.0528 Acc: 0.0772 Time: 0m 6s

Epoch 52/199
----------
train Loss: 0.1399 Acc: 0.9658 Time: 2m 46s
val Loss: 11.7683 Acc: 0.0825 Time: 0m 6s

Epoch 53/199
----------
train Loss: 0.2111 Acc: 0.9401 Time: 2m 46s
val Loss: 11.2892 Acc: 0.0825 Time: 0m 6s

Epoch 54/199
----------
train Loss: 0.3062 Acc: 0.9087 Time: 2m 46s
val Loss: 10.8536 Acc: 0.0656 Time: 0m 6s

Epoch 55/199
----------
train Loss: 0.3296 Acc: 0.9027 Time: 2m 47s
val Loss: 10.4900 Acc: 0.0730 Time: 0m 6s

Epoch 56/199
----------
train Loss: 0.2361 Acc: 0.9340 Time: 2m 46s
val Loss: 10.8132 Acc: 0.0709 Time: 0m 6s

Epoch 57/199
----------
train Loss: 0.1842 Acc: 0.9529 Time: 2m 47s
val Loss: 11.5818 Acc: 0.0624 Time: 0m 6s

Epoch 58/199
----------
train Loss: 0.1779 Acc: 0.9506 Time: 2m 45s
val Loss: 11.1591 Acc: 0.0698 Time: 0m 6s

Epoch 59/199
----------
train Loss: 0.1744 Acc: 0.9486 Time: 2m 46s
val Loss: 11.3297 Acc: 0.0847 Time: 0m 6s

Epoch 60/199
----------
train Loss: 0.2287 Acc: 0.9316 Time: 2m 45s
val Loss: 11.3057 Acc: 0.0698 Time: 0m 6s

Epoch 61/199
----------
train Loss: 0.2057 Acc: 0.9409 Time: 2m 46s
val Loss: 11.4712 Acc: 0.0688 Time: 0m 6s

Epoch 62/199
----------
train Loss: 0.2825 Acc: 0.9160 Time: 2m 46s
val Loss: 11.5232 Acc: 0.0698 Time: 0m 6s

Epoch 63/199
----------
train Loss: 0.2865 Acc: 0.9101 Time: 2m 45s
val Loss: 11.3645 Acc: 0.0646 Time: 0m 6s

Epoch 64/199
----------
train Loss: 0.2319 Acc: 0.9347 Time: 2m 46s
val Loss: 10.9853 Acc: 0.0772 Time: 0m 6s

Epoch 65/199
----------
train Loss: 0.1797 Acc: 0.9473 Time: 2m 46s
val Loss: 11.1300 Acc: 0.0783 Time: 0m 6s

Epoch 66/199
----------
train Loss: 0.1357 Acc: 0.9652 Time: 2m 46s
val Loss: 11.2078 Acc: 0.0857 Time: 0m 6s

Epoch 67/199
----------
train Loss: 0.1026 Acc: 0.9763 Time: 2m 47s
val Loss: 11.5920 Acc: 0.0772 Time: 0m 6s

Epoch 68/199
----------
train Loss: 0.1191 Acc: 0.9687 Time: 2m 46s
val Loss: 11.8193 Acc: 0.0815 Time: 0m 6s

Epoch 69/199
----------
train Loss: 0.0868 Acc: 0.9799 Time: 2m 45s
val Loss: 11.3214 Acc: 0.0783 Time: 0m 6s

Epoch 70/199
----------
train Loss: 0.0870 Acc: 0.9770 Time: 2m 45s
val Loss: 12.2018 Acc: 0.0709 Time: 0m 6s

Epoch 71/199
----------
train Loss: 0.1931 Acc: 0.9430 Time: 2m 46s
val Loss: 11.5722 Acc: 0.0741 Time: 0m 6s

Epoch 72/199
----------
train Loss: 0.1601 Acc: 0.9537 Time: 2m 45s
val Loss: 13.2912 Acc: 0.0709 Time: 0m 6s

Epoch 73/199
----------
train Loss: 0.3359 Acc: 0.8993 Time: 2m 46s
val Loss: 11.1617 Acc: 0.0656 Time: 0m 6s

Epoch 74/199
----------
train Loss: 0.2217 Acc: 0.9309 Time: 2m 46s
val Loss: 11.1376 Acc: 0.0529 Time: 0m 6s

Epoch 75/199
----------
train Loss: 0.1741 Acc: 0.9493 Time: 2m 45s
val Loss: 11.1473 Acc: 0.0741 Time: 0m 6s

Epoch 76/199
----------
train Loss: 0.2391 Acc: 0.9309 Time: 2m 47s
val Loss: 10.8280 Acc: 0.0741 Time: 0m 6s

Epoch 77/199
----------
train Loss: 0.1647 Acc: 0.9495 Time: 2m 47s
val Loss: 10.8620 Acc: 0.0667 Time: 0m 6s

Epoch 78/199
----------
train Loss: 0.0882 Acc: 0.9764 Time: 2m 46s
val Loss: 12.9628 Acc: 0.0571 Time: 0m 6s

Epoch 79/199
----------
train Loss: 0.0673 Acc: 0.9832 Time: 2m 47s
val Loss: 11.2955 Acc: 0.0815 Time: 0m 6s

Epoch 80/199
----------
train Loss: 0.0473 Acc: 0.9881 Time: 2m 45s
val Loss: 11.7334 Acc: 0.0868 Time: 0m 6s

Epoch 81/199
----------
train Loss: 0.0415 Acc: 0.9894 Time: 2m 46s
val Loss: 11.9521 Acc: 0.0836 Time: 0m 6s

Epoch 82/199
----------
train Loss: 0.0420 Acc: 0.9892 Time: 2m 46s
val Loss: 11.7551 Acc: 0.0836 Time: 0m 6s

Epoch 83/199
----------
train Loss: 0.0343 Acc: 0.9900 Time: 2m 46s
val Loss: 12.2095 Acc: 0.0720 Time: 0m 6s

Epoch 84/199
----------
train Loss: 0.0355 Acc: 0.9902 Time: 2m 46s
val Loss: 17.9678 Acc: 0.0455 Time: 0m 6s

Epoch 85/199
----------
train Loss: 0.0762 Acc: 0.9771 Time: 2m 46s
val Loss: 12.7392 Acc: 0.0698 Time: 0m 6s

Epoch 86/199
----------
train Loss: 0.1212 Acc: 0.9629 Time: 2m 47s
val Loss: 13.4803 Acc: 0.0582 Time: 0m 6s

Epoch 87/199
----------
train Loss: 0.1677 Acc: 0.9486 Time: 2m 46s
val Loss: 12.7376 Acc: 0.0624 Time: 0m 6s

Epoch 88/199
----------
train Loss: 0.4650 Acc: 0.8559 Time: 2m 47s
val Loss: 10.6509 Acc: 0.0677 Time: 0m 6s

Epoch 89/199
----------
train Loss: 0.2856 Acc: 0.9098 Time: 2m 47s
val Loss: 11.2772 Acc: 0.0593 Time: 0m 6s

Epoch 90/199
----------
train Loss: 0.1594 Acc: 0.9514 Time: 2m 47s
val Loss: 10.8819 Acc: 0.0667 Time: 0m 6s

Epoch 91/199
----------
train Loss: 0.2048 Acc: 0.9392 Time: 2m 45s
val Loss: 10.9760 Acc: 0.0614 Time: 0m 6s

Epoch 92/199
----------
train Loss: 0.0994 Acc: 0.9739 Time: 2m 45s
val Loss: 11.5573 Acc: 0.0688 Time: 0m 6s

Epoch 93/199
----------
train Loss: 0.2115 Acc: 0.9349 Time: 2m 46s
val Loss: 12.3761 Acc: 0.0529 Time: 0m 6s

Epoch 94/199
----------
train Loss: 0.1433 Acc: 0.9563 Time: 2m 45s
val Loss: 11.2955 Acc: 0.0825 Time: 0m 6s

Epoch 95/199
----------
train Loss: 0.0610 Acc: 0.9827 Time: 2m 45s
val Loss: 11.6232 Acc: 0.0825 Time: 0m 6s

Epoch 96/199
----------
train Loss: 0.0487 Acc: 0.9859 Time: 2m 47s
val Loss: 11.6406 Acc: 0.0720 Time: 0m 6s

Epoch 97/199
----------
train Loss: 0.1014 Acc: 0.9686 Time: 2m 47s
val Loss: 12.0203 Acc: 0.0667 Time: 0m 6s

Epoch 98/199
----------
train Loss: 0.1136 Acc: 0.9670 Time: 2m 48s
val Loss: 11.5569 Acc: 0.0783 Time: 0m 6s

Epoch 99/199
----------
train Loss: 0.0988 Acc: 0.9703 Time: 2m 46s
val Loss: 11.7552 Acc: 0.0836 Time: 0m 6s

Epoch 100/199
----------
train Loss: 0.0864 Acc: 0.9738 Time: 2m 46s
val Loss: 11.9590 Acc: 0.0783 Time: 0m 6s

Epoch 101/199
----------
train Loss: 0.0866 Acc: 0.9759 Time: 2m 47s
val Loss: 12.1071 Acc: 0.0825 Time: 0m 6s

Epoch 102/199
----------
train Loss: 0.1317 Acc: 0.9578 Time: 2m 46s
val Loss: 12.1521 Acc: 0.0698 Time: 0m 6s

Epoch 103/199
----------
train Loss: 0.1583 Acc: 0.9499 Time: 2m 45s
val Loss: 11.7781 Acc: 0.0720 Time: 0m 6s

Epoch 104/199
----------
train Loss: 0.0979 Acc: 0.9719 Time: 2m 46s
val Loss: 12.1074 Acc: 0.0772 Time: 0m 6s

Epoch 105/199
----------
train Loss: 0.0683 Acc: 0.9781 Time: 2m 45s
val Loss: 11.8538 Acc: 0.0772 Time: 0m 6s

Epoch 106/199
----------
train Loss: 0.1012 Acc: 0.9690 Time: 2m 47s
val Loss: 11.7520 Acc: 0.0815 Time: 0m 6s

Epoch 107/199
----------
train Loss: 0.1358 Acc: 0.9560 Time: 2m 45s
val Loss: 24.5194 Acc: 0.0529 Time: 0m 6s

Epoch 108/199
----------
train Loss: 0.4398 Acc: 0.8670 Time: 2m 45s
val Loss: 10.7842 Acc: 0.0593 Time: 0m 6s

Epoch 109/199
----------
train Loss: 0.1588 Acc: 0.9528 Time: 2m 47s
val Loss: 10.6987 Acc: 0.0794 Time: 0m 6s

Epoch 110/199
----------
train Loss: 0.0699 Acc: 0.9795 Time: 2m 47s
val Loss: 11.4489 Acc: 0.0762 Time: 0m 6s

Epoch 111/199
----------
train Loss: 0.0518 Acc: 0.9845 Time: 2m 46s
val Loss: 11.4950 Acc: 0.0794 Time: 0m 6s

Epoch 112/199
----------
train Loss: 0.0396 Acc: 0.9884 Time: 2m 47s
val Loss: 12.5029 Acc: 0.0730 Time: 0m 6s

Epoch 113/199
----------
train Loss: 0.1003 Acc: 0.9676 Time: 2m 45s
val Loss: 11.5720 Acc: 0.0741 Time: 0m 6s

Epoch 114/199
----------
train Loss: 0.0841 Acc: 0.9724 Time: 2m 47s
val Loss: 12.3733 Acc: 0.0730 Time: 0m 6s

Epoch 115/199
----------
train Loss: 0.0501 Acc: 0.9841 Time: 2m 47s
val Loss: 12.5260 Acc: 0.0772 Time: 0m 6s

Epoch 116/199
----------
train Loss: 0.0689 Acc: 0.9787 Time: 2m 47s
val Loss: 12.5352 Acc: 0.0720 Time: 0m 6s

Epoch 117/199
----------
train Loss: 0.0798 Acc: 0.9750 Time: 2m 46s
val Loss: 12.4840 Acc: 0.0741 Time: 0m 6s

Epoch 118/199
----------
train Loss: 0.2388 Acc: 0.9239 Time: 2m 46s
val Loss: 12.0678 Acc: 0.0571 Time: 0m 6s

Epoch 119/199
----------
train Loss: 0.1189 Acc: 0.9616 Time: 2m 46s
val Loss: 11.4813 Acc: 0.0794 Time: 0m 6s

Epoch 120/199
----------
train Loss: 0.1178 Acc: 0.9637 Time: 2m 46s
val Loss: 11.5148 Acc: 0.0794 Time: 0m 6s

Epoch 121/199
----------
train Loss: 0.0721 Acc: 0.9768 Time: 2m 47s
val Loss: 12.3177 Acc: 0.0730 Time: 0m 6s

Epoch 122/199
----------
train Loss: 0.0432 Acc: 0.9867 Time: 2m 48s
val Loss: 11.9371 Acc: 0.0741 Time: 0m 6s

Epoch 123/199
----------
train Loss: 0.0308 Acc: 0.9896 Time: 2m 49s
val Loss: 12.4937 Acc: 0.0794 Time: 0m 6s

Epoch 124/199
----------
train Loss: 0.0320 Acc: 0.9894 Time: 2m 45s
val Loss: 11.8992 Acc: 0.0688 Time: 0m 6s

Epoch 125/199
----------
train Loss: 0.0402 Acc: 0.9872 Time: 2m 45s
val Loss: 12.3704 Acc: 0.0868 Time: 0m 6s

Epoch 126/199
----------
train Loss: 0.0280 Acc: 0.9900 Time: 2m 45s
val Loss: 12.6141 Acc: 0.0794 Time: 0m 6s

Epoch 127/199
----------
train Loss: 0.0269 Acc: 0.9911 Time: 2m 46s
val Loss: 12.3049 Acc: 0.0762 Time: 0m 6s

Epoch 128/199
----------
train Loss: 0.0226 Acc: 0.9905 Time: 2m 46s
val Loss: 12.5782 Acc: 0.0762 Time: 0m 6s

Epoch 129/199
----------
train Loss: 0.0238 Acc: 0.9912 Time: 2m 45s
val Loss: 12.6876 Acc: 0.0730 Time: 0m 6s

Epoch 130/199
----------
train Loss: 0.1247 Acc: 0.9591 Time: 2m 46s
val Loss: 12.2929 Acc: 0.0646 Time: 0m 6s

Epoch 131/199
----------
train Loss: 0.1993 Acc: 0.9361 Time: 2m 46s
val Loss: 10.7169 Acc: 0.0709 Time: 0m 6s

Epoch 132/199
----------
train Loss: 0.1768 Acc: 0.9407 Time: 2m 46s
val Loss: 11.6649 Acc: 0.0624 Time: 0m 6s

Epoch 133/199
----------
train Loss: 0.1406 Acc: 0.9567 Time: 2m 46s
val Loss: 11.5506 Acc: 0.0772 Time: 0m 6s

Epoch 134/199
----------
train Loss: 0.1008 Acc: 0.9685 Time: 2m 46s
val Loss: 11.7249 Acc: 0.0730 Time: 0m 6s

Epoch 135/199
----------
train Loss: 0.0715 Acc: 0.9756 Time: 2m 46s
val Loss: 12.1409 Acc: 0.0688 Time: 0m 6s

Epoch 136/199
----------
train Loss: 0.0681 Acc: 0.9769 Time: 2m 46s
val Loss: 12.1633 Acc: 0.0772 Time: 0m 6s

Epoch 137/199
----------
train Loss: 0.0556 Acc: 0.9835 Time: 2m 46s
val Loss: 12.2010 Acc: 0.0815 Time: 0m 6s

Epoch 138/199
----------
train Loss: 0.0501 Acc: 0.9836 Time: 2m 46s
val Loss: 13.1082 Acc: 0.0751 Time: 0m 6s

Epoch 139/199
----------
train Loss: 0.0534 Acc: 0.9812 Time: 2m 46s
val Loss: 13.5240 Acc: 0.0847 Time: 0m 6s

Epoch 140/199
----------
train Loss: 0.0641 Acc: 0.9787 Time: 2m 47s
val Loss: 12.9939 Acc: 0.0677 Time: 0m 6s

Epoch 141/199
----------
train Loss: 0.1685 Acc: 0.9460 Time: 2m 47s
val Loss: 15.6878 Acc: 0.0402 Time: 0m 6s

Epoch 142/199
----------
train Loss: 0.1265 Acc: 0.9581 Time: 2m 46s
val Loss: 12.0243 Acc: 0.0646 Time: 0m 6s

Epoch 143/199
----------
train Loss: 0.1226 Acc: 0.9583 Time: 2m 47s
val Loss: 11.6328 Acc: 0.0688 Time: 0m 6s

Epoch 144/199
----------
train Loss: 0.0845 Acc: 0.9711 Time: 2m 46s
val Loss: 11.9953 Acc: 0.0667 Time: 0m 6s

Epoch 145/199
----------
train Loss: 0.1151 Acc: 0.9603 Time: 2m 46s
val Loss: 12.4355 Acc: 0.0698 Time: 0m 6s

Epoch 146/199
----------
train Loss: 0.0474 Acc: 0.9844 Time: 2m 47s
val Loss: 15.5728 Acc: 0.0444 Time: 0m 6s

Epoch 147/199
----------
train Loss: 0.0493 Acc: 0.9850 Time: 2m 45s
val Loss: 12.3067 Acc: 0.0762 Time: 0m 6s

Epoch 148/199
----------
train Loss: 0.0402 Acc: 0.9861 Time: 2m 48s
val Loss: 12.6724 Acc: 0.0730 Time: 0m 6s

Epoch 149/199
----------
train Loss: 0.0241 Acc: 0.9908 Time: 2m 46s
val Loss: 12.8210 Acc: 0.0794 Time: 0m 6s

Epoch 150/199
----------
train Loss: 0.0192 Acc: 0.9911 Time: 2m 46s
val Loss: 12.9653 Acc: 0.0847 Time: 0m 6s

Epoch 151/199
----------
train Loss: 0.0185 Acc: 0.9913 Time: 2m 47s
val Loss: 13.1477 Acc: 0.0783 Time: 0m 6s

Epoch 152/199
----------
train Loss: 0.0162 Acc: 0.9914 Time: 2m 46s
val Loss: 13.0547 Acc: 0.0815 Time: 0m 6s

Epoch 153/199
----------
train Loss: 0.0157 Acc: 0.9919 Time: 2m 47s
val Loss: 13.1442 Acc: 0.0794 Time: 0m 6s

Epoch 154/199
----------
train Loss: 0.0149 Acc: 0.9916 Time: 2m 47s
val Loss: 13.0178 Acc: 0.0825 Time: 0m 6s

Epoch 155/199
----------
train Loss: 0.0146 Acc: 0.9917 Time: 2m 45s
val Loss: 13.2372 Acc: 0.0868 Time: 0m 6s

Epoch 156/199
----------
train Loss: 0.0144 Acc: 0.9917 Time: 2m 47s
val Loss: 13.1048 Acc: 0.0794 Time: 0m 6s

Epoch 157/199
----------
train Loss: 0.0143 Acc: 0.9917 Time: 2m 46s
val Loss: 13.2554 Acc: 0.0847 Time: 0m 6s

Epoch 158/199
----------
train Loss: 0.0140 Acc: 0.9913 Time: 2m 47s
val Loss: 13.3671 Acc: 0.0815 Time: 0m 6s

Epoch 159/199
----------
train Loss: 0.2382 Acc: 0.9236 Time: 2m 46s
val Loss: 11.4707 Acc: 0.0677 Time: 0m 6s

Epoch 160/199
----------
train Loss: 0.4150 Acc: 0.8728 Time: 2m 47s
val Loss: 10.6705 Acc: 0.0635 Time: 0m 6s

Epoch 161/199
----------
train Loss: 0.2451 Acc: 0.9201 Time: 2m 46s
val Loss: 10.8037 Acc: 0.0730 Time: 0m 6s

Epoch 162/199
----------
train Loss: 0.0733 Acc: 0.9761 Time: 2m 47s
val Loss: 10.9393 Acc: 0.0741 Time: 0m 6s

Epoch 163/199
----------
train Loss: 0.0293 Acc: 0.9897 Time: 2m 46s
val Loss: 12.3503 Acc: 0.0772 Time: 0m 6s

Epoch 164/199
----------
train Loss: 0.0267 Acc: 0.9901 Time: 2m 46s
val Loss: 11.4731 Acc: 0.0783 Time: 0m 6s

Epoch 165/199
----------
train Loss: 0.0187 Acc: 0.9915 Time: 2m 46s
val Loss: 11.7100 Acc: 0.0815 Time: 0m 6s

Epoch 166/199
----------
train Loss: 0.0165 Acc: 0.9917 Time: 2m 47s
val Loss: 11.7614 Acc: 0.0815 Time: 0m 6s

Epoch 167/199
----------
train Loss: 0.0285 Acc: 0.9885 Time: 2m 46s
val Loss: 12.1898 Acc: 0.0709 Time: 0m 6s

Epoch 168/199
----------
train Loss: 0.0239 Acc: 0.9905 Time: 2m 47s
val Loss: 13.4019 Acc: 0.0794 Time: 0m 6s

Epoch 169/199
----------
train Loss: 0.0937 Acc: 0.9696 Time: 2m 47s
val Loss: 11.6028 Acc: 0.0709 Time: 0m 6s

Epoch 170/199
----------
train Loss: 0.0941 Acc: 0.9693 Time: 2m 47s
val Loss: 11.6232 Acc: 0.0688 Time: 0m 6s

Epoch 171/199
----------
train Loss: 0.0557 Acc: 0.9812 Time: 2m 46s
val Loss: 13.7361 Acc: 0.0646 Time: 0m 6s

Epoch 172/199
----------
train Loss: 0.0843 Acc: 0.9722 Time: 2m 46s
val Loss: 11.8420 Acc: 0.0847 Time: 0m 6s

Epoch 173/199
----------
train Loss: 0.1047 Acc: 0.9676 Time: 2m 46s
val Loss: 11.2040 Acc: 0.0720 Time: 0m 6s

Epoch 174/199
----------
train Loss: 0.0468 Acc: 0.9849 Time: 2m 47s
val Loss: 11.8213 Acc: 0.0624 Time: 0m 6s

Epoch 175/199
----------
train Loss: 0.0269 Acc: 0.9897 Time: 2m 47s
val Loss: 13.1627 Acc: 0.0688 Time: 0m 6s

Epoch 176/199
----------
train Loss: 0.0207 Acc: 0.9911 Time: 2m 46s
val Loss: 12.3845 Acc: 0.0730 Time: 0m 6s

Epoch 177/199
----------
train Loss: 0.0174 Acc: 0.9918 Time: 2m 46s
val Loss: 12.8632 Acc: 0.0709 Time: 0m 6s

Epoch 178/199
----------
train Loss: 0.0164 Acc: 0.9914 Time: 2m 47s
val Loss: 12.7093 Acc: 0.0698 Time: 0m 6s

Epoch 179/199
----------
train Loss: 0.0149 Acc: 0.9920 Time: 2m 48s
val Loss: 12.9577 Acc: 0.0709 Time: 0m 6s

Epoch 180/199
----------
train Loss: 0.0154 Acc: 0.9915 Time: 2m 47s
val Loss: 13.1454 Acc: 0.0730 Time: 0m 6s

Epoch 181/199
----------
train Loss: 0.0799 Acc: 0.9728 Time: 2m 46s
val Loss: 12.5256 Acc: 0.0667 Time: 0m 6s

Epoch 182/199
----------
train Loss: 0.1974 Acc: 0.9360 Time: 2m 46s
val Loss: 10.6568 Acc: 0.0720 Time: 0m 6s

Epoch 183/199
----------
train Loss: 0.2662 Acc: 0.9174 Time: 2m 47s
val Loss: 10.5179 Acc: 0.0815 Time: 0m 6s

Epoch 184/199
----------
train Loss: 0.0656 Acc: 0.9765 Time: 2m 46s
val Loss: 11.8831 Acc: 0.0730 Time: 0m 6s

Epoch 185/199
----------
train Loss: 0.0430 Acc: 0.9867 Time: 2m 46s
val Loss: 11.7909 Acc: 0.0762 Time: 0m 6s

Epoch 186/199
----------
train Loss: 0.0298 Acc: 0.9888 Time: 2m 46s
val Loss: 11.9705 Acc: 0.0868 Time: 0m 6s

Epoch 187/199
----------
train Loss: 0.0207 Acc: 0.9908 Time: 2m 47s
val Loss: 12.0552 Acc: 0.0804 Time: 0m 6s

Epoch 188/199
----------
train Loss: 0.0877 Acc: 0.9717 Time: 2m 46s
val Loss: 11.9347 Acc: 0.0730 Time: 0m 6s

Epoch 189/199
----------
train Loss: 0.1386 Acc: 0.9541 Time: 2m 45s
val Loss: 11.2207 Acc: 0.0720 Time: 0m 6s

Epoch 190/199
----------
train Loss: 0.0803 Acc: 0.9737 Time: 2m 46s
val Loss: 11.5400 Acc: 0.0656 Time: 0m 6s

Epoch 191/199
----------
train Loss: 0.0368 Acc: 0.9853 Time: 2m 47s
val Loss: 12.2822 Acc: 0.0677 Time: 0m 6s

Epoch 192/199
----------
train Loss: 0.0575 Acc: 0.9799 Time: 2m 47s
val Loss: 12.0157 Acc: 0.0582 Time: 0m 6s

Epoch 193/199
----------
train Loss: 0.0321 Acc: 0.9882 Time: 2m 45s
val Loss: 12.2591 Acc: 0.0646 Time: 0m 6s

Epoch 194/199
----------
train Loss: 0.0570 Acc: 0.9790 Time: 2m 47s
val Loss: 11.9296 Acc: 0.0688 Time: 0m 6s

Epoch 195/199
----------
train Loss: 0.0578 Acc: 0.9794 Time: 2m 46s
val Loss: 12.3644 Acc: 0.0582 Time: 0m 6s

Epoch 196/199
----------
train Loss: 0.0397 Acc: 0.9865 Time: 2m 46s
val Loss: 12.5478 Acc: 0.0656 Time: 0m 6s

Epoch 197/199
----------
train Loss: 0.0464 Acc: 0.9841 Time: 2m 45s
val Loss: 12.2663 Acc: 0.0635 Time: 0m 6s

Epoch 198/199
----------
train Loss: 0.0273 Acc: 0.9883 Time: 2m 47s
val Loss: 12.7503 Acc: 0.0741 Time: 0m 6s

Epoch 199/199
----------
train Loss: 0.0496 Acc: 0.9809 Time: 2m 46s
val Loss: 12.8409 Acc: 0.0688 Time: 0m 6s

Training complete in 573m 46s
Best val Acc: 0.087831
```

### result of 200 epoches, resnet 151, 40 classes, each class >= 500 samples
```text
Epoch 0/199
----------
train Loss: 3.4939 Acc: 0.1206 Time: 6m 40s
val Loss: 7.8154 Acc: 0.0217 Time: 0m 4s

Epoch 1/199
----------
train Loss: 3.3095 Acc: 0.1308 Time: 6m 41s
val Loss: 4.0907 Acc: 0.0450 Time: 0m 4s

Epoch 2/199
----------
train Loss: 3.2654 Acc: 0.1342 Time: 6m 40s
val Loss: 3.8879 Acc: 0.0367 Time: 0m 4s

Epoch 3/199
----------
train Loss: 3.2849 Acc: 0.1338 Time: 6m 41s
val Loss: 3.8035 Acc: 0.0317 Time: 0m 4s

Epoch 4/199
----------
train Loss: 3.4017 Acc: 0.1196 Time: 6m 40s
val Loss: 3.7909 Acc: 0.0383 Time: 0m 4s

Epoch 5/199
----------
train Loss: 3.2390 Acc: 0.1367 Time: 6m 37s
val Loss: 3.9625 Acc: 0.0333 Time: 0m 4s

Epoch 6/199
----------
train Loss: 3.1820 Acc: 0.1394 Time: 6m 38s
val Loss: 3.9632 Acc: 0.0383 Time: 0m 4s

Epoch 7/199
----------
train Loss: 3.1170 Acc: 0.1436 Time: 6m 40s
val Loss: 3.6707 Acc: 0.0667 Time: 0m 4s

Epoch 8/199
----------
train Loss: 3.0662 Acc: 0.1471 Time: 6m 37s
val Loss: 5.3176 Acc: 0.0650 Time: 0m 4s

Epoch 9/199
----------
train Loss: 3.1216 Acc: 0.1408 Time: 6m 39s
val Loss: 3.4817 Acc: 0.0567 Time: 0m 4s

Epoch 10/199
----------
train Loss: 3.0187 Acc: 0.1512 Time: 6m 40s
val Loss: 3.5302 Acc: 0.0683 Time: 0m 4s

Epoch 11/199
----------
train Loss: 3.1347 Acc: 0.1416 Time: 6m 40s
val Loss: 3.4014 Acc: 0.0650 Time: 0m 4s

Epoch 12/199
----------
train Loss: 2.9985 Acc: 0.1548 Time: 6m 35s
val Loss: 3.4672 Acc: 0.0750 Time: 0m 4s

Epoch 13/199
----------
train Loss: 2.9641 Acc: 0.1559 Time: 6m 36s
val Loss: 4.2783 Acc: 0.0800 Time: 0m 4s

Epoch 14/199
----------
train Loss: 2.9050 Acc: 0.1601 Time: 6m 41s
val Loss: 3.7943 Acc: 0.0717 Time: 0m 4s

Epoch 15/199
----------
train Loss: 2.9801 Acc: 0.1563 Time: 6m 38s
val Loss: 3.2485 Acc: 0.0883 Time: 0m 4s

Epoch 16/199
----------
train Loss: 2.9285 Acc: 0.1609 Time: 6m 40s
val Loss: 3.1707 Acc: 0.0700 Time: 0m 4s

Epoch 17/199
----------
train Loss: 2.8394 Acc: 0.1688 Time: 6m 39s
val Loss: 3.1730 Acc: 0.0983 Time: 0m 4s

Epoch 18/199
----------
train Loss: 2.8056 Acc: 0.1763 Time: 6m 38s
val Loss: 3.5139 Acc: 0.0650 Time: 0m 4s

Epoch 19/199
----------
train Loss: 2.8910 Acc: 0.1642 Time: 6m 38s
val Loss: 3.1884 Acc: 0.1033 Time: 0m 4s

Epoch 20/199
----------
train Loss: 2.7823 Acc: 0.1770 Time: 6m 41s
val Loss: 3.1158 Acc: 0.1233 Time: 0m 4s

Epoch 21/199
----------
train Loss: 2.7582 Acc: 0.1794 Time: 6m 40s
val Loss: 3.0696 Acc: 0.1200 Time: 0m 4s

Epoch 22/199
----------
train Loss: 2.7027 Acc: 0.1871 Time: 6m 40s
val Loss: 3.5182 Acc: 0.0867 Time: 0m 4s

Epoch 23/199
----------
train Loss: 2.6595 Acc: 0.1933 Time: 6m 37s
val Loss: 3.1995 Acc: 0.0967 Time: 0m 4s

Epoch 24/199
----------
train Loss: 2.6244 Acc: 0.1981 Time: 6m 41s
val Loss: 3.2488 Acc: 0.1033 Time: 0m 4s

Epoch 25/199
----------
train Loss: 2.5982 Acc: 0.2020 Time: 6m 41s
val Loss: 3.4360 Acc: 0.1267 Time: 0m 4s

Epoch 26/199
----------
train Loss: 2.5878 Acc: 0.2023 Time: 6m 39s
val Loss: 3.2651 Acc: 0.0883 Time: 0m 4s

Epoch 27/199
----------
train Loss: 2.5743 Acc: 0.2062 Time: 6m 40s
val Loss: 3.4338 Acc: 0.0967 Time: 0m 4s

Epoch 28/199
----------
train Loss: 2.5046 Acc: 0.2158 Time: 6m 38s
val Loss: 4.2744 Acc: 0.0683 Time: 0m 4s

Epoch 29/199
----------
train Loss: 2.8421 Acc: 0.1739 Time: 6m 38s
val Loss: 3.2021 Acc: 0.1300 Time: 0m 4s

Epoch 30/199
----------
train Loss: 2.6200 Acc: 0.1999 Time: 6m 39s
val Loss: 3.2642 Acc: 0.1033 Time: 0m 4s

Epoch 31/199
----------
train Loss: 2.4714 Acc: 0.2239 Time: 6m 38s
val Loss: 3.2653 Acc: 0.1217 Time: 0m 4s

Epoch 32/199
----------
train Loss: 2.4045 Acc: 0.2337 Time: 6m 40s
val Loss: 3.3549 Acc: 0.1167 Time: 0m 4s

Epoch 33/199
----------
train Loss: 2.3023 Acc: 0.2536 Time: 6m 39s
val Loss: 3.5103 Acc: 0.1267 Time: 0m 4s

Epoch 34/199
----------
train Loss: 2.2773 Acc: 0.2614 Time: 6m 40s
val Loss: 4.3042 Acc: 0.0917 Time: 0m 4s

Epoch 35/199
----------
train Loss: 2.4539 Acc: 0.2350 Time: 6m 39s
val Loss: 3.9089 Acc: 0.0850 Time: 0m 4s

Epoch 36/199
----------
train Loss: 2.9060 Acc: 0.1660 Time: 6m 40s
val Loss: 3.0842 Acc: 0.1083 Time: 0m 4s

Epoch 37/199
----------
train Loss: 2.5770 Acc: 0.2091 Time: 6m 41s
val Loss: 3.2050 Acc: 0.1150 Time: 0m 4s

Epoch 38/199
----------
train Loss: 2.4052 Acc: 0.2385 Time: 6m 37s
val Loss: 3.1982 Acc: 0.1267 Time: 0m 4s

Epoch 39/199
----------
train Loss: 2.2203 Acc: 0.2730 Time: 6m 37s
val Loss: 3.4448 Acc: 0.1267 Time: 0m 4s

Epoch 40/199
----------
train Loss: 2.1574 Acc: 0.2919 Time: 6m 37s
val Loss: 4.1054 Acc: 0.1133 Time: 0m 4s

Epoch 41/199
----------
train Loss: 2.1512 Acc: 0.2954 Time: 6m 38s
val Loss: 3.8738 Acc: 0.1350 Time: 0m 4s

Epoch 42/199
----------
train Loss: 1.9895 Acc: 0.3365 Time: 6m 41s
val Loss: 4.0805 Acc: 0.1217 Time: 0m 4s

Epoch 43/199
----------
train Loss: 1.9160 Acc: 0.3599 Time: 6m 41s
val Loss: 4.5581 Acc: 0.0867 Time: 0m 4s

Epoch 44/199
----------
train Loss: 2.7755 Acc: 0.1916 Time: 6m 41s
val Loss: 3.1244 Acc: 0.1117 Time: 0m 4s

Epoch 45/199
----------
train Loss: 2.3063 Acc: 0.2719 Time: 6m 37s
val Loss: 3.5948 Acc: 0.1033 Time: 0m 4s

Epoch 46/199
----------
train Loss: 1.9813 Acc: 0.3428 Time: 6m 39s
val Loss: 3.8226 Acc: 0.1167 Time: 0m 4s

Epoch 47/199
----------
train Loss: 1.7279 Acc: 0.4101 Time: 6m 39s
val Loss: 4.7998 Acc: 0.1000 Time: 0m 4s

Epoch 48/199
----------
train Loss: 1.7226 Acc: 0.4208 Time: 6m 39s
val Loss: 4.8905 Acc: 0.1250 Time: 0m 4s

Epoch 49/199
----------
train Loss: 1.6066 Acc: 0.4587 Time: 6m 38s
val Loss: 5.4428 Acc: 0.1000 Time: 0m 4s

Epoch 50/199
----------
train Loss: 1.2800 Acc: 0.5676 Time: 6m 37s
val Loss: 5.6222 Acc: 0.1133 Time: 0m 4s

Epoch 51/199
----------
train Loss: 1.6692 Acc: 0.4516 Time: 6m 39s
val Loss: 5.7873 Acc: 0.1233 Time: 0m 4s

Epoch 52/199
----------
train Loss: 1.1831 Acc: 0.6010 Time: 6m 40s
val Loss: 6.5513 Acc: 0.1033 Time: 0m 4s

Epoch 53/199
----------
train Loss: 1.1546 Acc: 0.6202 Time: 6m 39s
val Loss: 6.1205 Acc: 0.1217 Time: 0m 4s

Epoch 54/199
----------
train Loss: 1.1589 Acc: 0.6258 Time: 6m 41s
val Loss: 6.8071 Acc: 0.1217 Time: 0m 4s

Epoch 55/199
----------
train Loss: 0.8425 Acc: 0.7324 Time: 6m 42s
val Loss: 7.2696 Acc: 0.1067 Time: 0m 4s

Epoch 56/199
----------
train Loss: 0.9523 Acc: 0.7019 Time: 6m 37s
val Loss: 6.8457 Acc: 0.1067 Time: 0m 4s

Epoch 57/199
----------
train Loss: 0.4879 Acc: 0.8522 Time: 6m 39s
val Loss: 7.7371 Acc: 0.1233 Time: 0m 4s

Epoch 58/199
----------
train Loss: 1.2375 Acc: 0.6388 Time: 6m 37s
val Loss: 6.3449 Acc: 0.1283 Time: 0m 4s

Epoch 59/199
----------
train Loss: 0.5078 Acc: 0.8471 Time: 6m 39s
val Loss: 7.4511 Acc: 0.1050 Time: 0m 4s

Epoch 60/199
----------
train Loss: 0.3921 Acc: 0.8915 Time: 6m 39s
val Loss: 8.2750 Acc: 0.1083 Time: 0m 4s

Epoch 61/199
----------
train Loss: 0.4524 Acc: 0.8702 Time: 6m 39s
val Loss: 8.4438 Acc: 0.1167 Time: 0m 4s

Epoch 62/199
----------
train Loss: 0.3947 Acc: 0.8916 Time: 6m 37s
val Loss: 8.4017 Acc: 0.1133 Time: 0m 4s

Epoch 63/199
----------
train Loss: 0.4517 Acc: 0.8727 Time: 6m 41s
val Loss: 8.0020 Acc: 0.1200 Time: 0m 4s

Epoch 64/199
----------
train Loss: 0.4967 Acc: 0.8580 Time: 6m 36s
val Loss: 7.8056 Acc: 0.1233 Time: 0m 4s

Epoch 65/199
----------
train Loss: 0.6992 Acc: 0.8044 Time: 6m 36s
val Loss: 7.4002 Acc: 0.1017 Time: 0m 4s

Epoch 66/199
----------
train Loss: 0.2680 Acc: 0.9298 Time: 6m 41s
val Loss: 8.0337 Acc: 0.1150 Time: 0m 4s

Epoch 67/199
----------
train Loss: 0.2786 Acc: 0.9250 Time: 6m 37s
val Loss: 8.3608 Acc: 0.1200 Time: 0m 4s

Epoch 68/199
----------
train Loss: 0.2699 Acc: 0.9319 Time: 6m 36s
val Loss: 8.4952 Acc: 0.1033 Time: 0m 4s

Epoch 69/199
----------
train Loss: 0.7649 Acc: 0.7893 Time: 6m 39s
val Loss: 7.4164 Acc: 0.0983 Time: 0m 4s

Epoch 70/199
----------
train Loss: 0.2462 Acc: 0.9378 Time: 6m 37s
val Loss: 8.1010 Acc: 0.1133 Time: 0m 4s

Epoch 71/199
----------
train Loss: 0.3148 Acc: 0.9174 Time: 6m 37s
val Loss: 7.9690 Acc: 0.1033 Time: 0m 4s

Epoch 72/199
----------
train Loss: 0.2440 Acc: 0.9372 Time: 6m 38s
val Loss: 8.3060 Acc: 0.1017 Time: 0m 4s

Epoch 73/199
----------
train Loss: 0.2451 Acc: 0.9401 Time: 6m 38s
val Loss: 8.2104 Acc: 0.1117 Time: 0m 4s

Epoch 74/199
----------
train Loss: 0.2357 Acc: 0.9376 Time: 6m 36s
val Loss: 8.1197 Acc: 0.1133 Time: 0m 4s

Epoch 75/199
----------
train Loss: 0.1141 Acc: 0.9760 Time: 6m 39s
val Loss: 8.6027 Acc: 0.1283 Time: 0m 4s

Epoch 76/199
----------
train Loss: 0.4543 Acc: 0.8748 Time: 6m 39s
val Loss: 7.8672 Acc: 0.1167 Time: 0m 4s

Epoch 77/199
----------
train Loss: 0.2435 Acc: 0.9375 Time: 6m 40s
val Loss: 7.7213 Acc: 0.1283 Time: 0m 4s

Epoch 78/199
----------
train Loss: 0.3016 Acc: 0.9175 Time: 6m 36s
val Loss: 7.8467 Acc: 0.0967 Time: 0m 4s

Epoch 79/199
----------
train Loss: 0.1572 Acc: 0.9617 Time: 6m 36s
val Loss: 8.4628 Acc: 0.1133 Time: 0m 4s

Epoch 80/199
----------
train Loss: 0.5814 Acc: 0.8382 Time: 6m 41s
val Loss: 7.4925 Acc: 0.1100 Time: 0m 4s

Epoch 81/199
----------
train Loss: 0.1549 Acc: 0.9621 Time: 6m 37s
val Loss: 8.1172 Acc: 0.1383 Time: 0m 4s

Epoch 82/199
----------
train Loss: 0.1616 Acc: 0.9606 Time: 6m 36s
val Loss: 8.3559 Acc: 0.1133 Time: 0m 4s

Epoch 83/199
----------
train Loss: 0.1346 Acc: 0.9674 Time: 6m 38s
val Loss: 7.9662 Acc: 0.1183 Time: 0m 4s

Epoch 84/199
----------
train Loss: 0.1100 Acc: 0.9738 Time: 6m 39s
val Loss: 8.5696 Acc: 0.1183 Time: 0m 4s

Epoch 85/199
----------
train Loss: 0.2646 Acc: 0.9293 Time: 6m 40s
val Loss: 8.0628 Acc: 0.1217 Time: 0m 4s

Epoch 86/199
----------
train Loss: 0.4923 Acc: 0.8622 Time: 6m 36s
val Loss: 7.5431 Acc: 0.0967 Time: 0m 4s

Epoch 87/199
----------
train Loss: 0.2077 Acc: 0.9416 Time: 6m 38s
val Loss: 7.8322 Acc: 0.1183 Time: 0m 4s

Epoch 88/199
----------
train Loss: 0.0706 Acc: 0.9865 Time: 6m 36s
val Loss: 7.7299 Acc: 0.1333 Time: 0m 4s

Epoch 89/199
----------
train Loss: 0.1540 Acc: 0.9608 Time: 6m 36s
val Loss: 8.2101 Acc: 0.1067 Time: 0m 4s

Epoch 90/199
----------
train Loss: 0.0666 Acc: 0.9857 Time: 6m 40s
val Loss: 8.4663 Acc: 0.1183 Time: 0m 4s

Epoch 91/199
----------
train Loss: 0.1540 Acc: 0.9578 Time: 6m 40s
val Loss: 8.5836 Acc: 0.1067 Time: 0m 4s

Epoch 92/199
----------
train Loss: 0.2130 Acc: 0.9391 Time: 6m 37s
val Loss: 8.1027 Acc: 0.1283 Time: 0m 4s

Epoch 93/199
----------
train Loss: 0.2570 Acc: 0.9272 Time: 6m 40s
val Loss: 8.3909 Acc: 0.1083 Time: 0m 4s

Epoch 94/199
----------
train Loss: 0.1540 Acc: 0.9576 Time: 6m 37s
val Loss: 8.0835 Acc: 0.1267 Time: 0m 4s

Epoch 95/199
----------
train Loss: 0.2629 Acc: 0.9243 Time: 6m 42s
val Loss: 8.1206 Acc: 0.0917 Time: 0m 4s

Epoch 96/199
----------
train Loss: 0.2162 Acc: 0.9396 Time: 6m 36s
val Loss: 7.9173 Acc: 0.1233 Time: 0m 4s

Epoch 97/199
----------
train Loss: 0.0655 Acc: 0.9844 Time: 6m 38s
val Loss: 7.8589 Acc: 0.1300 Time: 0m 4s

Epoch 98/199
----------
train Loss: 0.1212 Acc: 0.9683 Time: 6m 37s
val Loss: 8.4767 Acc: 0.1117 Time: 0m 4s

Epoch 99/199
----------
train Loss: 0.1574 Acc: 0.9564 Time: 6m 38s
val Loss: 8.3353 Acc: 0.1183 Time: 0m 4s

Epoch 100/199
----------
train Loss: 0.1772 Acc: 0.9488 Time: 6m 37s
val Loss: 8.0979 Acc: 0.1183 Time: 0m 4s

Epoch 101/199
----------
train Loss: 0.0577 Acc: 0.9857 Time: 6m 38s
val Loss: 8.4522 Acc: 0.1317 Time: 0m 4s

Epoch 102/199
----------
train Loss: 0.1382 Acc: 0.9623 Time: 6m 38s
val Loss: 8.4304 Acc: 0.1083 Time: 0m 4s

Epoch 103/199
----------
train Loss: 0.1358 Acc: 0.9610 Time: 6m 41s
val Loss: 8.7592 Acc: 0.1217 Time: 0m 4s

Epoch 104/199
----------
train Loss: 0.2221 Acc: 0.9382 Time: 6m 38s
val Loss: 8.2680 Acc: 0.1050 Time: 0m 4s

Epoch 105/199
----------
train Loss: 0.0669 Acc: 0.9828 Time: 6m 38s
val Loss: 9.0557 Acc: 0.1200 Time: 0m 4s

Epoch 106/199
----------
train Loss: 0.1473 Acc: 0.9576 Time: 6m 39s
val Loss: 8.9061 Acc: 0.1100 Time: 0m 4s

Epoch 107/199
----------
train Loss: 0.3504 Acc: 0.8961 Time: 6m 40s
val Loss: 8.0602 Acc: 0.0967 Time: 0m 4s

Epoch 108/199
----------
train Loss: 0.1734 Acc: 0.9493 Time: 6m 40s
val Loss: 8.4954 Acc: 0.1167 Time: 0m 4s

Epoch 109/199
----------
train Loss: 0.1007 Acc: 0.9723 Time: 6m 40s
val Loss: 8.5099 Acc: 0.1283 Time: 0m 4s

Epoch 110/199
----------
train Loss: 0.0941 Acc: 0.9742 Time: 6m 40s
val Loss: 8.9728 Acc: 0.1050 Time: 0m 4s

Epoch 111/199
----------
train Loss: 0.1615 Acc: 0.9511 Time: 6m 38s
val Loss: 8.5283 Acc: 0.0917 Time: 0m 4s

Epoch 112/199
----------
train Loss: 0.2426 Acc: 0.9288 Time: 6m 38s
val Loss: 8.1559 Acc: 0.1200 Time: 0m 4s

Epoch 113/199
----------
train Loss: 0.1127 Acc: 0.9678 Time: 6m 37s
val Loss: 8.5399 Acc: 0.1183 Time: 0m 4s

Epoch 114/199
----------
train Loss: 0.2036 Acc: 0.9427 Time: 6m 42s
val Loss: 8.5207 Acc: 0.0967 Time: 0m 4s

Epoch 115/199
----------
train Loss: 0.0526 Acc: 0.9854 Time: 6m 38s
val Loss: 8.4517 Acc: 0.1000 Time: 0m 4s

Epoch 116/199
----------
train Loss: 0.1988 Acc: 0.9433 Time: 6m 38s
val Loss: 8.3920 Acc: 0.0967 Time: 0m 4s

Epoch 117/199
----------
train Loss: 0.0516 Acc: 0.9853 Time: 6m 42s
val Loss: 8.7236 Acc: 0.1200 Time: 0m 4s

Epoch 118/199
----------
train Loss: 0.0362 Acc: 0.9883 Time: 6m 39s
val Loss: 8.5541 Acc: 0.1117 Time: 0m 4s

Epoch 119/199
----------
train Loss: 0.0883 Acc: 0.9734 Time: 6m 36s
val Loss: 9.1517 Acc: 0.1100 Time: 0m 4s

Epoch 120/199
----------
train Loss: 0.1026 Acc: 0.9700 Time: 6m 38s
val Loss: 9.0146 Acc: 0.1050 Time: 0m 4s

Epoch 121/199
----------
train Loss: 0.2120 Acc: 0.9374 Time: 6m 41s
val Loss: 8.4447 Acc: 0.1133 Time: 0m 4s

Epoch 122/199
----------
train Loss: 0.2293 Acc: 0.9301 Time: 6m 41s
val Loss: 8.6467 Acc: 0.1067 Time: 0m 4s

Epoch 123/199
----------
train Loss: 0.1369 Acc: 0.9590 Time: 6m 38s
val Loss: 8.7230 Acc: 0.0967 Time: 0m 4s

```